In [11]:
import os
import math
import nltk
import re
import pandas as pd

from enum import Enum

In [12]:

class PennTreebank(Enum):
    CC = 1 # Coordinating conjunction
    CD = 2 # Cardinal number
    DT = 3 # Determiner
    EX = 4 # Existential there
    FW = 5 # Foreign word
    IN = 6 # Preposition or subordinating conjunction
    JJ = 7 # Adjective
    JJR = 8 # Adjective, comparative
    JJS = 9 # Adjective, superlative
    LS = 10 # List item marker
    MD = 11 # Modal
    NN = 12 # Noun, singular or mass
    NNS = 13 # Noun, plural
    NNP = 14 # Proper noun, singular
    NNPS = 15 # Proper noun, plural
    PDT = 16 # Predeterminer
    POS = 17 # Possessive ending
    PRP = 18 # Personal pronoun
    # PRP$ = 19 # Possessive pronoun
    RB = 20 # Adverb
    RBR = 21 # Adverb, comparative
    RBS = 22 # Adverb, superlative
    RP = 23 # Particle
    SYM = 24 # Symbol
    TO = 25 # to
    UH = 26 # Interjection
    VB = 27 # Verb, base form
    VBD = 28 # Verb, past tense
    VBG = 29 # Verb, gerund or present participle
    VBN = 30 # Verb, past participle
    VBP = 31 # Verb, non-3rd person singular present
    VBZ = 32 # Verb, 3rd person singular present
    WDT = 33 # Wh-determiner
    WP = 34 # Wh-pronoun
    # WP$ = 35 # Possessive wh-pronoun
    WRB = 36 # Wh-adverb


In [13]:
def read_document(filename: str) -> str:
    """Reads text from file with name: filename
    """
    with open(filename, "r", encoding="utf8") as file:
        return file.read()


In [62]:
def build_train_test_per_fold(
        datasets: list,
        classes: str,
        data_path: str,
        filenames: list,
        fold: int,
        max_train_files: int = None,
        max_test_files: int = None,
) -> list:
    """Prepares the train-test split of the files to be used later
    """
    trains = {c: [] for c in classes}
    tests = {c: [] for c in classes}
    
    for c in classes:
        for filename in filenames[c]:
            file_path = f"{data_path}/{c}/{filename}"
            if filename[2] == str(fold):
                tests[c].append(file_path)
                trains[c].append(file_path)
            else:
                trains[c].append(file_path)

    max_trains = len(trains.items()) \
        if not max_train_files \
        else max_train_files

    max_tests = len(tests.items()) \
        if not max_test_files \
        else max_test_files

    datasets.append({
        'train': {
            c: d[:max_trains] for (c, d) in trains.items()
        },
        'test': {
            c: d[:max_tests] for (c, d) in tests.items()
        }
    })

    return datasets


In [63]:
def make_datasets(
    data_path: str,
    n_folds: int = 1,
    max_train_files: int = None,
    max_test_files: int = None,
):
    """Makes the train and test datasets
    """
    classes = os.listdir(data_path)
    filenames = {
        c: sorted(os.listdir(f"{data_path}/{c}/")) 
        for c in classes
    }
    
    datasets = []
    for fold in range(n_folds):
        build_train_test_per_fold(
            datasets=datasets,
            classes=classes,
            data_path=data_path,
            filenames=filenames,
            fold=fold,
            max_train_files=max_train_files,
            max_test_files=max_test_files,
        )
    
    return datasets

In [67]:
def append_tag_pattern(
        condition: bool,
        tag_pattern: list,
        first_word: list,
        second_word: list,
) -> list:
    if condition:
        tag_pattern.append(
            "".join(first_word) + " " + "".join(second_word)
        )
        return tag_pattern

In [68]:
def check_turney_patterns(
        postag_1,
        postag_2,
        postag_3,
):
    patterns = []

    patterns.append(
        postag_1[1]== PennTreebank.JJ.name and
        (postag_2[1] == PennTreebank.NN.name
         or postag_2[1] == PennTreebank.NNS.name)
    )

    patterns.append(
        (postag_1[1] == PennTreebank.RB.name or
         postag_1[1] == PennTreebank.RBR.name or
         postag_1[1] == PennTreebank.RBS.name) and
        (postag_2[1] == PennTreebank.JJ.name and
         postag_3[1] != PennTreebank.NN.name and
         postag_3[1] != PennTreebank.NNS.name)
    )

    patterns.append(
        postag_1[1] == PennTreebank.JJ.name and
        postag_2[1] == PennTreebank.JJ.name and
        postag_3[1] != PennTreebank.NN.name and
        postag_3[1] != PennTreebank.NNS.name)

    patterns.append(
        (postag_1[1] == PennTreebank.NN.name or
         postag_1[1] == PennTreebank.NNS.name) and
        (postag_2[1] == PennTreebank.JJ.name and
         postag_3[1] != PennTreebank.NN.name and
         postag_3[1] != PennTreebank.NNS.name)
    )

    patterns.append(
        (postag_1[1] == PennTreebank.RB.name or
         postag_1[1] == PennTreebank.RBR.name or
         postag_1[1] == PennTreebank.RBS.name) and
        (postag_2[1] == PennTreebank.VB.name or
         postag_2[1] == PennTreebank.VBD.name or
         postag_2[1] == PennTreebank.VBN.name or
         postag_2[1] == PennTreebank.VBG.name)
    )

    return any(patterns)


In [69]:
def find_pattern(postag):
    tag_pattern = []
    for k in range(len(postag)-2):
        append_tag_pattern(
            condition=check_turney_patterns(
                postag_1=postag[k],
                postag_2=postag[k+1],
                postag_3=postag[k+2],
            ),
            tag_pattern=tag_pattern,
            first_word=postag[k][0],
            second_word=postag[k+1][0],
        )
    return tag_pattern

In [70]:
def near_operator(phrase, word, text):
    try:
        string = word+r'\W+(?:\w+\W+){0,500}?'+phrase+r'|'+phrase+r'\W+(?:\w+\W+){0,500}?'+word
        freq_phrase_near_word = (len(re.findall(string,text)))
        return freq_phrase_near_word
    except:
        return 0

In [102]:
class Turney(object):

    def __init__(self, datasets, n_folds: int = 1):
        self.datasets = datasets
        self.pos_phrases_hits = []
        self.neg_phrases_hits = []
        self.pos_hits = 0.01
        self.neg_hits = 0.01
        self.pos_hits_init = 0.01
        self.neg_hits_init = 0.01
        self.accuracy = 0
        self._positive_class = "pos"
        self._negative_class = "neg"
        self._classes = [self._positive_class, self._negative_class]
        self._great_word = "great"
        self._poor_word = "poor"
        self.sentiments = {}
        self.confussion_matrix = pd.DataFrame({
                "ActualPos": [0,0],
                "ActualNeg": [0,0],
            }, index=["PredPos", "PredNeg"]
        )
        self.n_folds = n_folds

    def train(self, phrases: list, n_fold: int = 0):
        self.pos_phrases_hits = [self.pos_hits_init] * len(phrases)
        self.neg_phrases_hits = [self.neg_hits_init] * len(phrases)
        self.pos_hits = self.pos_hits_init
        self.neg_hits = self.neg_hits_init

        # print("Callibrating the model:")
        for train_klass in self._classes:
            # print(f"Class: {train_klass}")
            for k, file in enumerate(self.datasets[n_fold]['train'][train_klass]):
                # print(f"Processing file: {k}")
                
                txt_file = read_document(file)
                for ind, phrase in enumerate(phrases):
                    self.pos_phrases_hits[ind] += near_operator(
                        phrase=phrase,
                        word=self._great_word,
                        text=txt_file
                    )
                    self.neg_phrases_hits[ind] += near_operator(
                        phrase=phrase,
                        word=self._poor_word,
                        text=txt_file
                    )
                    self.pos_hits += txt_file.count(self._great_word)
                    self.neg_hits += txt_file.count(self._poor_word)
                
                # print(f"So far: pos_hits = {self.pos_hits} | neg_hits = {self.neg_hits}")
        
    def evaluate(self):
        n_fold = 0
        for test_klass in self._classes:
            for i, data in enumerate(self.datasets[n_fold]['test'][test_klass]):
                print(f"{test_klass.title()} Document: {i}")
                text = read_document(data)
                doc_tokens = find_pattern(nltk.pos_tag(nltk.word_tokenize(text)))

                self.train(doc_tokens)

                so = self._predict_so()

                is_negative = (test_klass==self._negative_class)
                self._update_confusion_matrix(so, is_negative)
                
                self.sentiments.update({
                    data: so
                })
                print(f"\tPredicted = {so} ({'pos' if so>0 else 'neg'}) | Actual = {test_klass}")

        print("Final Confussion Matrix")
        print(self.confussion_matrix)
            
    def _predict_so(self):
        polarities = [
            math.log((self.pos_phrases_hits[i] * self.neg_hits) /
                     (self.neg_phrases_hits[i] * self.pos_hits), 2)
            for i in range(len(self.pos_phrases_hits))
        ]

        so = sum(polarities)/len(polarities)        
        return so

    def _update_confusion_matrix(self, predict, is_negative: bool):
        if predict > 0 and (not is_negative):
            self.confussion_matrix.loc["PredPos", "ActualPos"] += 1
        if predict < 0 and is_negative:
            self.confussion_matrix.loc["PredNeg", "ActualNeg"] += 1
        if predict > 0 and is_negative:
            self.confussion_matrix.loc["PredPos", "ActualNeg"] += 1
        if predict < 0 and not is_negative:
            self.confussion_matrix.loc["PredNeg", "ActualPos"] += 1

In [106]:
DATA_PATH = './imdb1/'
datasets = make_datasets(DATA_PATH, max_train_files=500, max_test_files=20)

In [107]:
turney = Turney(datasets)

In [108]:
turney.evaluate()

Pos Document: 0
	Predicted = 2.6286907943573437 (pos) | Actual = pos
Pos Document: 1
	Predicted = -1.057635406910201 (neg) | Actual = pos
Pos Document: 2
	Predicted = -1.1282492532119346 (neg) | Actual = pos
Pos Document: 3
	Predicted = 4.06120015223865 (pos) | Actual = pos
Pos Document: 4
	Predicted = -1.419650896879345 (neg) | Actual = pos
Pos Document: 5
	Predicted = 3.312462363927977 (pos) | Actual = pos
Pos Document: 6
	Predicted = 1.7591797493144823 (pos) | Actual = pos
Pos Document: 7
	Predicted = -0.8784895565305946 (neg) | Actual = pos
Pos Document: 8
	Predicted = -1.0964236412326798 (neg) | Actual = pos
Pos Document: 9
	Predicted = -1.592839664868666 (neg) | Actual = pos
Pos Document: 10
	Predicted = -1.3384503029467367 (neg) | Actual = pos
Pos Document: 11
	Predicted = 2.6263187774413583 (pos) | Actual = pos
Pos Document: 12
	Predicted = 4.793041422477484 (pos) | Actual = pos
Pos Document: 13
	Predicted = -1.0238202785200987 (neg) | Actual = pos
Pos Document: 14
	Predicted =